# Importações e Instalações de Biblioteca



In [ ]:
!pip install 'aif360[Reductions,inFairness]'
!pip install xlrd


  Using cached adversarial_robustness_toolbox-1.17.1-py3-none-any.whl.metadata (11 kB)
  Using cached igraph-0.11.4-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.9 kB)
  Using cached fairlearn-0.10.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached jupyter-1.0.0-py2.py3-none-any.whl.metadata (995 bytes)
  Using cached rpy2-3.5.16.tar.gz (220 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Unable to determine R home: [Errno 2] No such file or directory: 'R'
      cffi mode is CFFI_MODE.ANY
      Looking for R home with: R RHOME
      Unable to determine R home: [Errno 2] No such file or directory: 'R'
      R home found: None
      Error: rpy2 in API mode cannot be built without R in the PATH or R_HOME defined. Correct this or force ABI mode-only by defining the env

In [6]:

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import ast
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [7]:
# Fairness lib
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.postprocessing import EqOddsPostprocessing

2024-04-07 22:29:28.838488: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-07 22:29:33.094812: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/tuao/anaconda3/envs/aif360/lib/python3.11/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


# Métodos de Verificação de Vieses

## Funções

In [8]:
def justica_exps(base, attr_set, df, parametros, modelo, categorical_features, label, fvr_classes, prt_attrs, priv_classes):

  # criando data set no formato adequado (classe Standard Dataset)
  df2model = StandardDataset(df,
                            label_name=label,
                            favorable_classes=fvr_classes,
                            protected_attribute_names=prt_attrs,
                            privileged_classes=priv_classes)

  i = 0
  results = []
  for param_set in parametros:
    result_grid = {}
    print(type(param_set))
    print(param_set)

    # Estabelecendo conjunto de parametros para o modelo
    # Criando modelo
    if modelo == 'gbt':
      clf = GradientBoostingClassifier(n_estimators = param_set['n_estimators'],
                                        min_samples_split = param_set['min_samples_split'],
                                        learning_rate = param_set['learning_rate'],
                                        max_depth = param_set['max_depth'],
                                        random_state = param_set['random_state'])
    elif modelo == 'rf':
      clf = RandomForestClassifier(n_estimators = param_set['n_estimators'],
                                  min_samples_split = param_set['min_samples_split'],
                                  criterion = param_set['criterion'],
                                  max_depth = param_set['max_depth'],
                                  random_state = param_set['random_state'])
    else:
      print("Modelos suportados atualmente: 'gbt' e 'rf'. ")

    # Separando teste e treino
    df2_train, df2_test = df2model.split([0.7], shuffle=True)

    # Fitando o modelo na base de treino
    model = clf.fit(df2_train.features, df2_train.labels.ravel())

    # Formatando base de teste
    x_df2_test = df2_test.features
    y_df2_test = df2_test.labels.ravel()

    # Criando copia da base para predição
    dataset = df2_test
    dataset_pred = dataset.copy()
    dataset_pred.labels = model.predict(df2_test.features)

    # # Definindo as populações privilegiadas e não-privilegiadas
    # # Sexo
    # ## Recuperando o indice
    # privileged_sex   = np.where(categorical_names['Sexo'] == 'male')[0]
    # unprivileged_sex = np.where(categorical_names['Sexo'] == 'female')[0]
    # ## Estabelecendo formato, seguindo o formato da documentacao das classes ClassificationMetric and BinaryLabelDatasetMetric
    # privileged_groups   = [{'Sexo' : privileged_sex}]
    # unprivileged_groups = [{'Sexo' : unprivileged_sex}]

    # Criando classes de populacoes
    classified_metric_sex = ClassificationMetric(dataset,
                                                 dataset_pred,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)

    metric_pred_sex = BinaryLabelDatasetMetric(dataset_pred,
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)

    # # Idade
    # ## Recuperando o indice
    # privileged_faixa_etaria   = np.where(categorical_names['faixa_etaria'] == 'adulto')[0]
    # unprivileged_faixa_etaria = np.where(categorical_names['faixa_etaria'] == 'jovem')[0]
    # ## Estabelecendo formato, seguindo o formato da documentacao das classes ClassificationMetric and BinaryLabelDatasetMetric
    # privileged_groups_fe   = [{'faixa_etaria' : privileged_faixa_etaria}]
    # unprivileged_groups_fe = [{'faixa_etaria' : unprivileged_faixa_etaria}]

    # Criando classes de populacoes
    classified_metric_faixa_etaria = ClassificationMetric(dataset,
                                                          dataset_pred,
                                                          unprivileged_groups=unprivileged_groups_fe,
                                                          privileged_groups=privileged_groups_fe)

    metric_pred_faixa_etaria = BinaryLabelDatasetMetric(dataset_pred,
                                                        unprivileged_groups=unprivileged_groups_fe,
                                                        privileged_groups=privileged_groups_fe)

    # Submetendo modelo ao 10-fold cross validation (fitting)
    # obtendo acurácia
    scores1 = cross_val_score(clf, df2_train.features, df2_train.labels.ravel(), cv=10)
    print("%0.2f de acuracia com desvio padrão de %0.2f" % (scores1.mean(), scores1.std()))
    # obtendo f1_score
    scores2 = cross_val_score(clf, df2_train.features, df2_train.labels.ravel(), cv=10,scoring='f1_macro')
    print("%0.2f de f1 com desvio padrão de %0.2f" % (scores2.mean(), scores2.std()))
    # obtendo precisão
    scores3 = cross_val_score(clf, df2_train.features, df2_train.labels.ravel(), cv=10,scoring='precision')
    print("%0.2f de precisão com desvio padrão de %0.2f" % (scores3.mean(), scores3.std()))

    # Obtendo o score no teste
    model = clf.fit(df2_train.features, df2_train.labels.ravel())
    score4 = model.score(x_df2_test, y_df2_test)
    print("%0.2f de score no teste" % (score4))

    # Métricas de justica algoritmica
    print('STATISTICAL PARITY DIFFERENCE')
    spd_sexo = metric_pred_sex.statistical_parity_difference()
    spd_faixa_etaria = metric_pred_faixa_etaria.statistical_parity_difference()

    print(f'Atributo sexo: {spd_sexo}')
    print(f'Atributo faixa_etaria: {spd_faixa_etaria}')

    print('')

    print('DISPARATE IMPACT')
    di_sexo = metric_pred_sex.disparate_impact()
    di_faixa_etaria = metric_pred_faixa_etaria.disparate_impact()

    print(f'Atributo sexo: {di_sexo}')
    print(f'Atributo faixa_etaria: {di_faixa_etaria}')

    print(' ')

    print('AVERAGE ODDS DIFFERENCE')
    aod_sexo = classified_metric_sex.average_odds_difference()
    aod_faixa_etaria = classified_metric_faixa_etaria.average_odds_difference()

    print(f'Atributo sexo: {aod_sexo}')
    print(f'Atributo faixa_etaria: {aod_faixa_etaria}')

    print(' ')

    print('EQUAL OPPORTUNITY DIFFERENCE')

    eod_sexo = classified_metric_sex.equal_opportunity_difference()
    eod_faixa_etaria = classified_metric_faixa_etaria.equal_opportunity_difference()
    print(f'Atributo sexo: {eod_sexo}')
    print(f'Atributo faixa_etaria: {eod_faixa_etaria}')

    # collecting results
    result_grid['index'] = i
    result_grid['base'] = base
    result_grid['attr_set'] = attr_set
    result_grid['modelo'] = modelo
    result_grid['param_set'] = str(param_set)
    result_grid['10fold-acuracia'] = scores1.mean()
    result_grid['10fold-f1'] = scores2.mean()
    result_grid['10fold-precisao'] = scores3.mean()
    result_grid['test-score'] = score4
    result_grid['spd_sexo'] = spd_sexo
    result_grid['spd_faixa_etaria'] = spd_faixa_etaria
    result_grid['di_sexo'] = di_sexo
    result_grid['di_faixa_etaria'] = di_faixa_etaria
    result_grid['aod_sexo'] = aod_sexo
    result_grid['aod_faixa_etaria'] = aod_faixa_etaria
    result_grid['eod_sexo'] = eod_sexo
    result_grid['eod_faixa_etaria'] = eod_faixa_etaria

    results.append(result_grid)
    i += 1
    print(result_grid)
    print("")
  return result_grid

# Lendo e processando a base de dados



Descrição dos dados no link: https://www.kaggle.com/datasets/uciml/default-of-credit-card-clients-dataset




In [9]:
base = 'taiwan'
# Carregar o conjunto de dados Default of Credit Card Clients
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"

# Carregar o conjunto de dados, ignorando as primeiras linhas que são metadados
taiwan = pd.read_excel(url, skiprows=1)

# Renomear a primeira coluna para "ID" e apaga-la
taiwan.rename(columns={'ID': 'Client_ID'}, inplace=True)
taiwan = taiwan.drop('Client_ID', axis=1)


print(taiwan.head())

   LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  PAY_5  PAY_6  BILL_AMT1  BILL_AMT2  BILL_AMT3  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  PAY_AMT4  PAY_AMT5  PAY_AMT6  default payment next month
0      20000    2          2         1   24      2      2     -1     -1     -2     -2       3913       3102        689          0          0          0         0       689         0         0         0         0                           1
1     120000    2          2         2   26     -1      2      0      0      0      2       2682       1725       2682       3272       3455       3261         0      1000      1000      1000         0      2000                           1
2      90000    2          2         2   34      0      0      0      0      0      0      29239      14027      13559      14331      14948      15549      1518      1500      1000      1000      1000      5000                           0
3      50000    2          2         1  

**Produzindo categorias para faixa etária**

In [10]:
taiwan.loc[taiwan['AGE']<=24, 'faixa_etaria'] = 'jovem'
taiwan.loc[taiwan['AGE']>24, 'faixa_etaria'] = 'adulto'

**Produzindo Categoria Binária de sexo**

In [11]:

taiwan.rename(columns={'default payment next month' : 'class'}, inplace=True)
taiwan.loc[taiwan['SEX'].isin([1]), 'Sexo'] = 'male'
taiwan.loc[taiwan['SEX'].isin([2]), 'Sexo'] = 'female'

In [12]:
#separando rótulo (variável dependente y)
taiwan_y = taiwan['class']

## Codificando categorias

In [13]:
categorical_features = [i for i in taiwan.columns if taiwan.dtypes[i] in ['object', 'category']]
print(categorical_features)

['faixa_etaria', 'Sexo']


In [14]:
categorical_names = {}
encoders = {}

for feature in categorical_features:
  le = LabelEncoder()
  le.fit(taiwan[feature])

  taiwan[feature] = le.transform(taiwan[feature])

  categorical_names[feature] = le.classes_
  encoders[feature] = le

In [15]:
taiwan = taiwan.astype(int)
taiwan.dtypes
# todos devem estar codificados em int64

LIMIT_BAL       int64
SEX             int64
EDUCATION       int64
MARRIAGE        int64
AGE             int64
PAY_0           int64
PAY_2           int64
PAY_3           int64
PAY_4           int64
PAY_5           int64
PAY_6           int64
BILL_AMT1       int64
BILL_AMT2       int64
BILL_AMT3       int64
BILL_AMT4       int64
BILL_AMT5       int64
BILL_AMT6       int64
PAY_AMT1        int64
PAY_AMT2        int64
PAY_AMT3        int64
PAY_AMT4        int64
PAY_AMT5        int64
PAY_AMT6        int64
class           int64
faixa_etaria    int64
Sexo            int64
dtype: object

In [16]:
taiwan.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,class,faixa_etaria,Sexo
0,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1,1,0
1,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1,0,0
2,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0,0,0
3,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0,0,0
4,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0,0,1


In [17]:
# Definições sobre os dados para avaliação de justiça
taiwan.at[1, 'class'] = 0
label_name ='class'
protected_attribute_names = ['Sexo', 'faixa_etaria']
favorable_classes = np.where(taiwan['class'] == 1)[0]

In [18]:
# Definindo as populações privilegiadas e não-privilegiadas
# Sexo
## Recuperando o indice
privileged_sex   = np.where(categorical_names['Sexo'] == 'male')[0]
unprivileged_sex = np.where(categorical_names['Sexo'] == 'female')[0]
## Estabelecendo formato, seguindo o formato da documentacao das classes ClassificationMetric and BinaryLabelDatasetMetric
privileged_groups   = [{'Sexo' : privileged_sex}]
unprivileged_groups = [{'Sexo' : unprivileged_sex}]

# Idade
## Recuperando o indice
privileged_faixa_etaria   = np.where(categorical_names['faixa_etaria'] == 'adulto')[0]
unprivileged_faixa_etaria = np.where(categorical_names['faixa_etaria'] == 'jovem')[0]
## Estabelecendo formato, seguindo o formato da documentacao das classes ClassificationMetric and BinaryLabelDatasetMetric
privileged_groups_fe   = [{'faixa_etaria' : privileged_faixa_etaria}]
unprivileged_groups_fe = [{'faixa_etaria' : unprivileged_faixa_etaria}]


# Dados privilegiados
privileged_classes = [privileged_sex, privileged_faixa_etaria]

# Preparando experimentos

In [19]:
np.random.seed(2024)

In [20]:
list_gbt_params = []
for estimators in range(100, 500, 100):
  for samples in range(50, 300, 50):
    for learning in [0.1, 0.2, 0.3, 0.4, 0.5]:
      for depth in range(2, 8, 1):
        gbt_p = {}
        RANDOM_STATE = 2024
        gbt_p['n_estimators'] = estimators
        gbt_p['min_samples_split'] = samples
        gbt_p['learning_rate'] = learning
        gbt_p['max_depth'] = depth
        gbt_p['random_state'] = RANDOM_STATE
        list_gbt_params.append(gbt_p)

# print(*(x for x in list_gbt_params), sep='\n')
print("Total de {} conjuntos de parâmetros.".format(len(list_gbt_params)))

Total de 600 conjuntos de parâmetros.


In [21]:
list_rf_params = []
for estimators in range(100, 500, 100):
  for samples in range(50, 300, 50):
    for learning in ['gini', 'entropy']:
      for depth in range(2, 8, 1):
        rf_p = {}
        RANDOM_STATE = 2024
        rf_p['n_estimators'] = estimators
        rf_p['min_samples_split'] = samples
        rf_p['criterion'] = learning
        rf_p['max_depth'] = depth
        rf_p['random_state'] = RANDOM_STATE
        list_rf_params.append(rf_p)

# print(*(x for x in list__params), sep='\n')
print("Total de {} conjuntos de parâmetros.".format(len(list_rf_params)))

Total de 240 conjuntos de parâmetros.


# Toda a base

## GBT

In [22]:
exp1_all_gbt = justica_exps(base, 'toda-a-base', taiwan, list_gbt_params, 'gbt', categorical_features, 'class', favorable_classes, protected_attribute_names, privileged_classes)

<class 'dict'>
{'n_estimators': 100, 'min_samples_split': 50, 'learning_rate': 0.1, 'max_depth': 2, 'random_state': 2024}


0.82 de acuracia com desvio padrão de 0.01
0.68 de f1 com desvio padrão de 0.02
0.68 de precisão com desvio padrão de 0.03
0.82 de score no teste
STATISTICAL PARITY DIFFERENCE
Atributo sexo: 0.014684920116410316
Atributo faixa_etaria: -0.05725865867169111

DISPARATE IMPACT
Atributo sexo: 1.0168311776718857
Atributo faixa_etaria: 0.9354057404994464
 
AVERAGE ODDS DIFFERENCE
Atributo sexo: 0.002353170555773332
Atributo faixa_etaria: -0.040195646240009986
 
EQUAL OPPORTUNITY DIFFERENCE
Atributo sexo: 0.007837405932351382
Atributo faixa_etaria: -0.01920411899776453
{'index': 0, 'base': 'taiwan', 'attr_set': 'toda-a-base', 'modelo': 'gbt', 'param_set': "{'n_estimators': 100, 'min_samples_split': 50, 'learning_rate': 0.1, 'max_depth': 2, 'random_state': 2024}", '10fold-acuracia': 0.822904761904762, '10fold-f1': 0.6805911119151055, '10fold-precisao': 0.6759669363174663, 'test-score': 0.8171111111111111, 'spd_sexo': 0.014684920116410316, 'spd_faixa_etaria': -0.05725865867169111, 'di_sexo': 1.0

KeyboardInterrupt: 

## RF


In [ ]:
exp2_all_rf = justica_exps(base, 'toda-a-base', taiwan, list_rf_params, 'rf', categorical_features, 'class', favorable_classes, protected_attribute_names, privileged_classes)